<a href="https://colab.research.google.com/github/esdrasantos/statistical-concepts-and-practice/blob/main/cstfm_nao_parametricos_qui_quadrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qui Quadrado

In [95]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
import numpy as np
import pandas as pd

# Exemplo 1

In [97]:
amazon = pd.read_excel('/content/drive/MyDrive/Curso de Estatística 2.0/Arquivos Excel/Dados3_CursoEstatistica.xlsx')

# Em formato longo
amazon.head()

,Categoria,Renda,Quantidade
0,Camera,25000<,26
1,Camera,25-49999,38
2,Camera,50-74999,76
3,Camera,75-99999,57
4,Camera,100000>,50


In [98]:
# Tabela, um variavel em eixo x, outra em y
# x: Categoria, y: Quantidade
tabela = pd.crosstab(amazon['Categoria'], amazon['Renda'])
tabela

# Esse só funciona para super longo (quando dado repete observação)

Renda,100000>,25-49999,25000<,50-74999,75-99999
Categoria,,,,,
Camera,1,1,1,1,1
Celular,1,1,1,1,1


In [99]:
# x: Categoria, y: Renda, z: Quantidadde agrupada pela Soma
tabela = pd.crosstab(amazon['Categoria'], amazon['Renda'], amazon['Quantidade'], aggfunc='sum')
tabela

Renda,100000>,25-49999,25000<,50-74999,75-99999
Categoria,,,,,
Camera,50,38,26,76,57
Celular,53,76,72,73,34


In [100]:
from scipy.stats import chi2_contingency

chi2_contingency(tabela)

(stat, p, grau_liberdade, valor_esperado) = chi2_contingency(tabela)

print(f'estatística do teste: {stat}')
print(f'p-valor: {p}')
print(f'grau_liberdade: {grau_liberdade}')
print(f'valor_esperado: {valor_esperado}')

estatística do teste: 33.9247848239719
p-valor: 7.721305846006497e-07
grau_liberdade: 4
valor_esperado: [[45.83963964 50.73513514 43.61441441 66.31171171 40.4990991 ]
 [57.16036036 63.26486486 54.38558559 82.68828829 50.5009009 ]]


In [101]:
# Interpretação

alfa = 0.05

if p < alfa:
  print('Hipótese nula rejeitada')
  print('Há relação entra as variáveis')
else:
  print('Hipótese nula não rejeitada')
  print('Não há relação entre as variáveis')

Hipótese nula rejeitada
Há relação entra as variáveis


In [102]:
df_valor_esperado = pd.DataFrame(valor_esperado, index=tabela.index, columns=tabela.columns)

df_valor_esperado

Renda,100000>,25-49999,25000<,50-74999,75-99999
Categoria,,,,,
Camera,45.83964,50.735135,43.614414,66.311712,40.499099
Celular,57.16036,63.264865,54.385586,82.688288,50.500901


In [103]:
# Onde está a relação de dependência?

residuos_de_pearson = (tabela - valor_esperado) / np.sqrt(valor_esperado)
residuos_de_pearson

Renda,100000>,25-49999,25000<,50-74999,75-99999
Categoria,,,,,
Camera,0.614484,-1.787924,-2.667185,1.18974,2.592895
Celular,-0.550280,1.601114,2.388506,-1.06543,-2.321978


In [104]:
mask = abs(residuos_de_pearson)> 2
significativos = residuos_de_pearson[mask]
significativos

Renda,100000>,25-49999,25000<,50-74999,75-99999
Categoria,,,,,
Camera,NaN,NaN,-2.667185,NaN,2.592895
Celular,NaN,NaN,2.388506,NaN,-2.321978


In [105]:
# Para linha e coluna:
print('São significativas as relações entre')
for linha in residuos_de_pearson.index:
  for coluna in residuos_de_pearson.columns:
    if abs(residuos_de_pearson.loc[linha, coluna]) > 2:
      print(f'Linha: {linha}, Coluna: {coluna}')

São significativas as relações entre
Linha: Camera, Coluna: 25000<
Linha: Camera, Coluna: 75-99999
Linha: Celular, Coluna: 25000<
Linha: Celular, Coluna: 75-99999


# Exemplo 2

In [106]:
camisas = pd.read_excel('/content/drive/MyDrive/Curso de Estatística 2.0/Arquivos Excel/Dados3_CursoEstatistica.xlsx', sheet_name=1)

camisas.head()

,Tamanho,Estilo,Quantidade
0,Pequeno,Botões,18
1,Pequeno,Polo,27
2,Pequeno,Estampa,36
3,Medio,Botões,65
4,Medio,Polo,82


In [107]:
tabela_2 = pd.crosstab(camisas['Tamanho'], camisas['Estilo'], camisas['Quantidade'], aggfunc='sum')
print(tabela_2)

Estilo    Botões  Estampa  Polo
Tamanho                        
Grande       103       22    65
Medio         65       28    82
Pequeno       18       36    27


In [108]:
(stat, p, grau_liberdade, valor_esperado) = chi2_contingency(tabela_2)

print(f'estatística do teste: {stat}')
print(f'p-valor: {p}')
print(f'grau_liberdade: {grau_liberdade}')
print(f'valor_esperado: {valor_esperado}')

estatística do teste: 53.3447025743991
p-valor: 7.217571232852959e-11
grau_liberdade: 4
valor_esperado: [[79.23766816 36.6367713  74.12556054]
 [72.98206278 33.74439462 68.2735426 ]
 [33.78026906 15.61883408 31.60089686]]


In [109]:
# Interpretação:

if p < alfa:
  print('Hipótese nula rejeitada')
  print('Há relação entra as variáveis')
else:
  print('Hipótese nula não rejeitada')
  print('Não há relação entre as variáveis')

Hipótese nula rejeitada
Há relação entra as variáveis


In [110]:
pd.DataFrame(valor_esperado, index=tabela_2.index, columns=tabela_2.columns)

Estilo,Botões,Estampa,Polo
Tamanho,,,
Grande,79.237668,36.636771,74.125561
Medio,72.982063,33.744395,68.273543
Pequeno,33.780269,15.618834,31.600897


In [111]:
# Residuos Ajustados de Pearson

residuos_de_pearson_ = (tabela_2 - valor_esperado) / np.sqrt(valor_esperado)
residuos_de_pearson_

Estilo,Botões,Estampa,Polo
Tamanho,,,
Grande,2.669459,-2.418169,-1.059926
Medio,-0.934345,-0.988880,1.661239
Pequeno,-2.715081,5.157090,-0.818451


In [112]:
mask = abs(residuos_de_pearson_)> 2
significativos = residuos_de_pearson_[mask]
significativos

Estilo,Botões,Estampa,Polo
Tamanho,,,
Grande,2.669459,-2.418169,NaN
Medio,NaN,NaN,NaN
Pequeno,-2.715081,5.157090,NaN


In [113]:
# Decisão
print('Comprar maior quantidade de camisa do tipo:')
for linha in residuos_de_pearson_.index:
  for coluna in residuos_de_pearson_.columns:
    if abs(residuos_de_pearson_.loc[linha, coluna]) > 2:
      print(f'Linha: {linha}, Coluna: {coluna}')

Comprar maior quantidade de camisa do tipo:
Linha: Grande, Coluna: Botões
Linha: Grande, Coluna: Estampa
Linha: Pequeno , Coluna: Botões
Linha: Pequeno , Coluna: Estampa


# Exemplo 3

In [114]:
contas = pd.read_excel('/content/drive/MyDrive/Curso de Estatística 2.0/Arquivos Excel/Dados3_CursoEstatistica.xlsx', sheet_name=2)

contas.head()

,Status,Canal,Quantidade
0,Fechada,Loja,48
1,Fechada,Envio,9
2,Fechada,Website,28
3,Corrente,Loja,200
4,Corrente,Envio,65


In [115]:
tabela_3 = pd.crosstab(contas['Status'], contas['Canal'], contas['Quantidade'], aggfunc='sum')
tabela_3

Canal,Envio,Loja,Website
Status,,,
Atraso,40,71,51
Corrente,65,200,118
Fechada,9,48,28


In [116]:
(stat, p, grau_liberdade, valor_esperado) = chi2_contingency(tabela_3)

print(f'estatística do teste: {stat}')
print(f'p-valor: {p}')
print(f'grau_liberdade: {grau_liberdade}')
print(f'valor_esperado: {valor_esperado}')

estatística do teste: 9.15831550035679
p-valor: 0.057261892663582774
grau_liberdade: 4
valor_esperado: [[ 29.31428571  82.02857143  50.65714286]
 [ 69.3047619  193.93174603 119.76349206]
 [ 15.38095238  43.03968254  26.57936508]]


In [118]:
# Interpretação

if p < alfa:
  print('Hipótese nula rejeitada')
  print('Há relação entra as variáveis')
else:
  print('Hipótese nula não rejeitada')
  print('Não há relação entre as variáveis')
  print('Logo, a relação entre o canal da compra e seu status não tem diferença significativa')

Hipótese nula não rejeitada
Não há relação entre as variáveis
Logo, a relação entre o canal da compra e seu status não tem diferença significativa
